## Interest Rate Sentiment Monitor

Instructions: 
1. For each of the four code cells below, click on the cell then hit Shift+Enter on your keyboard to run the code (or if on a mobile device, press 'play' button). 
2. Optionally, edit the comments in Code cell 2 to analyze different sentiments.

**Code cell 1:** Set up programming environment to use code to send prompts to OpenAI's cloud-hosted service.

In [14]:
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")


def llm_response(prompt):
    response = openai.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
   
    return response.choices[0].message.content

**Code cell 2:** Create a list of comments related to interest rates.

In [15]:
interest_rate_comments = [
    'The central bank raising interest rates will curb inflation.',
    'Higher interest rates have made it difficult for us to secure a mortgage.',
    'Interest rate hikes are slowing down the economy.',
    'The adjustment in interest rates is positively impacting the currency value.',
    'Rising interest rates are putting pressure on small businesses.'
]

interest_rate_comments

['The central bank raising interest rates will curb inflation.',
 'Higher interest rates have made it difficult for us to secure a mortgage.',
 'Interest rate hikes are slowing down the economy.',
 'The adjustment in interest rates is positively impacting the currency value.',
 'Rising interest rates are putting pressure on small businesses.']

**Code cell 3:** Classify the comments as positive or negative.

In [16]:
all_sentiments = []
for comment in interest_rate_comments:
    prompt = f'''
        Classify the following comment
        as having either a positive or
        negative sentiment regarding interest rates. State your answer
        as a single word, either "positive" or "negative" or "neutral:

        {comment}
        '''
    response = llm_response(prompt)
    all_sentiments.append(response)

all_sentiments

['positive', 'negative', 'negative', 'positive', 'negative']

**Code cell 4:** Count the number of positive and negative comments.

In [ ]:
num_positive = 0
num_negative = 0
num_neutral = 0 
total_comments = len(all_sentiments)
for sentiment in all_sentiments:
    if sentiment == 'positive':
        num_positive += 1
    elif sentiment == 'negative':
        num_negative += 1
    elif sentiment == 'neutral':
        num_negative += 1

# Calculate percentages
percent_positive = (num_positive / total_comments) * 100 if total_comments else 0
percent_negative = (num_negative / total_comments) * 100 if total_comments else 0
percent_neutral = (num_neutral / total_comments) * 100 if total_comments else 0

# Print counts and percentages
print(f"There are {num_positive} positive, {num_negative} negative, and {num_neutral} neutral comments about interest rates.")
print(f"Percentage of Positive Comments: {percent_positive:.2f}%")
print(f"Percentage of Negative Comments: {percent_negative:.2f}%")
print(f"Percentage of Neutral Comments: {percent_neutral:.2f}%")



There are 2 positive and 3 negative comments about interest rates.
There are 2 positive, 3 negative, and 0 neutral comments about interest rates.
Percentage of Positive Comments: 40.00%
Percentage of Negative Comments: 60.00%
Percentage of Neutral Comments: 0.00%


## RAG

In [ ]:
from datasets import load_dataset

dataset = load_dataset("infoslack/mistral-7b-arxiv-paper-chunked", split="train")

dataset

In [ ]:
dataset[0]

In [ ]:
data = dataset.to_pandas()

In [ ]:
data.head()

In [ ]:
docs = data[['chunk', 'source']]
docs.head()

In [ ]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              openai_api_key=openai_api_key)

url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

qdrant = Qdrant.from_documents(
    documents=documents,
    embedding=embeddings,
    url=url,
    collection_name="chatbot",
    api_key=api_key
)

In [ ]:
query = "What are the the intrest rate sentiments today?"
qdrant.similarity_search(query, k=3)

In [ ]:
def custom_prompt(query: str):
    results = qdrant.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.page_content for x in results])
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augment_prompt

In [ ]:
print(custom_prompt(query))

In [ ]:
prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

## Groq

In [ ]:
from langchain_groq import ChatGroq
chat = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

In [ ]:
prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)